# Consumptions and Energy Generation analysis

This notebook describes all data analysis on resource consumption and power generation.

Loading the necessary packages:

In [114]:
import pandas as pd
from datetime import date, timedelta

Set the corresponding variables for the data analysis:
- files for actual year data set and historical data
- year of analysis

In [115]:
actuals_file_name:str = "./../bin/" + "2023.csv"
hist_file_name:str = "./../bin/" + "history.csv"
results_folder_path:str = "./../bin/"
ANALYSIS_YEAR:int = date.today().year

Load the data frames from the csv files:

In [116]:
actuals_df = pd.read_csv(actuals_file_name, index_col='Date', parse_dates=['Date'])

Calculate Statistics per Year

In [117]:
stats_dict = {}


# Columns in the data frame
# Gas	Water	Power In	Power Out	Power Gen	Power PV used	Power used	Power Car Stephan	Power Car Heike	Power Car Wink	
for col in actuals_df.columns:
    print(col + "   ", end="")
# general consumption statistics
consumtion = round(actuals_df["Gas"][-1] - actuals_df["Gas"][0], 2)
stats_dict["Gasverbrauch (kWh)"] = consumtion
consumtion = round(actuals_df["Water"][-1] - actuals_df["Water"][0], 2)
stats_dict["Wasserverbrauch (qm)"] = consumtion
consumtion = round(actuals_df["Power In"][-1] - actuals_df["Power In"][0], 2)
stats_dict["Strombezug (kWh)"] = consumtion

# vehicle energy consumption
consumtion = round(actuals_df["Power Car Stephan"][-1] - actuals_df["Power Car Stephan"][0], 2)
stats_dict["Strom für V60 (kWh)"] = consumtion
consumtion = round(actuals_df["Power Car Heike"][-1] - actuals_df["Power Car Heike"][0], 2)
stats_dict["Strom für XC40 (kWh)"] = consumtion
consumtion = stats_dict["Strom für V60 (kWh)"] 
consumtion = consumtion + stats_dict["Strom für XC40 (kWh)"] 
consumtion = round(consumtion + (actuals_df["Power Car Wink"][-1] - actuals_df["Power Car Wink"][0]), 2)
stats_dict["Strom für alle Autos (kWh)"] = consumtion

# energy production statistics
stats_dict["Stromproduktion (kWh)"] = round(actuals_df["Power Gen"].sum(), 2)
stats_dict["Eigenverbrauch (kWh)"] = round(actuals_df["Power PV used"].sum(), 2)
stats_dict["Stromverbrauch (kWh)"] = round(actuals_df["Power used"].sum(), 2)
stats_dict["Max Stromproduktion per Tag (kWh)"] = round(actuals_df["Power Gen"].max(), 2)
stats_dict["Max Stromproduktion Tag"] = actuals_df["Power Gen"].idxmax()
stats_dict
# get a subset of the data frame where the power generation is not 0.0
actuals_with_pwr_df = actuals_df[actuals_df["Power PV used"] > 0.0]
stats_dict["Mittelwert Stromproduktion (kWh)"] = round(actuals_with_pwr_df["Power Gen"].mean(), 2)
stats_dict["Mittelwert Eigenverbrauch (kWh)"] = round(actuals_with_pwr_df["Power PV used"].mean(), 2)
stats_dict["Mittelwert Tagesverbrauch (kWh)"] = round(actuals_with_pwr_df["Power used"].mean(), 2)
stats_dict["Geschätzter Verbrauch pro Jahr (kWh)"] = round(actuals_with_pwr_df["Power used"].mean() * 356, 2)
stats_dict["Geschätzter Autarkiegrad (%)"] = round(stats_dict["Mittelwert Eigenverbrauch (kWh)"] /
                                                   stats_dict["Mittelwert Tagesverbrauch (kWh)"] * 100.0, 2)
stats_dict["Stromanteil Autos (%)"] = round(stats_dict["Strom für alle Autos (kWh)"] /
                                            stats_dict["Strombezug (kWh)"] * 100.0, 2)




Gas   Water   Power In   Power Out   Power Gen   Power PV used   Power used   Power Car Stephan   Power Car Heike   Power Car Wink   

Display the statistics per year

In [118]:
for keys, values in stats_dict.items():
    print(f"{keys}:    {values}")

Gasverbrauch (kWh):    277.0
Wasserverbrauch (qm):    50.0
Strombezug (kWh):    2254.0
Strom für V60 (kWh):    111.0
Strom für XC40 (kWh):    172.0
Strom für alle Autos (kWh):    376.0
Stromproduktion (kWh):    3360.65
Eigenverbrauch (kWh):    1513.95
Stromverbrauch (kWh):    1885.08
Max Stromproduktion per Tag (kWh):    53.0
Max Stromproduktion Tag:    2023-09-05 00:00:00
Mittelwert Stromproduktion (kWh):    23.83
Mittelwert Eigenverbrauch (kWh):    10.74
Mittelwert Tagesverbrauch (kWh):    13.37
Geschätzter Verbrauch pro Jahr (kWh):    4759.49
Geschätzter Autarkiegrad (%):    80.33
Stromanteil Autos (%):    16.68


get a dictionary with month start and end dates

In [119]:
def get_start_end_dates_of_month(year:int)->dict:
    """Calculates for that particular year all first days and last days of each month.
    Args:
        year (int): integer year to create the start and end dates of each month
    Returns:
        dict: a dictionary with keys for the month number and a list set of two
        dates, the start and end dates of that particular month
    """
    # date.today().year
    date_dict = {}
    for month in range(1,13):
        if month != 12:
            dict_value = []
            start_date = date(year, month, 1)
            end_date = date(year, month + 1, 1)
            end_date = end_date - timedelta(days=1)
            dict_value.append(start_date)
            dict_value.append(end_date)
            date_dict[month] = dict_value
        else:
            dict_value = []
            start_date = date(year, 12, 1)
            end_date = date(year, 12, 31)
            dict_value.append(start_date)
            dict_value.append(end_date)
            date_dict[month] = dict_value
    return date_dict

Calculate the monthy statistics(mainly based on the daily power data). The rest of the actuals file isn't tracked clearly.

In [120]:

date_dict = get_start_end_dates_of_month(ANALYSIS_YEAR)

mth_stats_diary = {}
for key, dates in date_dict.items():
    mth_df = actuals_df[dates[0]:dates[1]]
    if mth_df.empty == False:
        mth_dict = {}
        # energy production statistics
        mth_dict["Stromproduktion (kWh)"] = round(mth_df["Power Gen"].sum(), 2)
        mth_dict["Eigenverbrauch (kWh)"] = round(mth_df["Power PV used"].sum(), 2)
        mth_dict["Stromverbrauch (kWh)"] = round(mth_df["Power used"].sum(), 2)
        mth_dict["Max Stromproduktion per Tag (kWh)"] = round(mth_df["Power Gen"].max(), 2)
        mth_dict["Max Stromproduktion Tag"] = mth_df["Power Gen"].idxmax()
        mth_stats_diary[dates[0].strftime('%B')] = mth_dict

#for key, value in mth_stats_diary.items():
#    print(key)
#    print(value)

# Creates DataFrame. 
mth_stats_df = pd.DataFrame(mth_stats_diary) 
  
# Print the data 
mth_stats_df = mth_stats_df.transpose()
result_file = results_folder_path + str(ANALYSIS_YEAR) + "_mthstats_csv"
mth_stats_df.to_csv(result_file)